In [1]:
%load_ext autoreload
%autoreload all

In [7]:
import sys
import os
import getpass

if getpass.getuser() == "benjamin.lebrun":
    sys.path.append("/home/mila/b/benjamin.lebrun/genparse")
    os.environ["HF_HOME"] = os.path.join(os.environ["SCRATCH"], "hf_cache")
    print("HF cache set; path updated")

if getpass.getuser() == "hooverjac":
    sys.path.append(os.path.join(os.environ["HOME"], "proj-genparse/genparse"))
    os.environ["HF_HOME"] = os.environ["SCRATCH"]
    print("HF cache set; path updated", getpass.getuser())

if getpass.getuser() == "j":
    sys.path.append("/Users/j/CHI/repos/genparse")
    print("j on mac")

import nest_asyncio
nest_asyncio.apply()

HF cache set; path updated hooverjac


In [8]:
prompt = """
    You have access to a political survey data table named "data", which includes the following columns:
    - "age" (integer)
    - "gender" ("male" or "female"),
    - "year" (integer)
    - "state_color" ("blue" or "red")
    - "zipcode" (integer)
    - "vote" ("democrat" or "republican") 
    - "race_ethnicity" ("white", "black", or "latino").

    Q: Write a SQL query that shows individuals' age and gender, for people over 50 years old.
    A: SELECT age, gender FROM data WHERE age>50 </s>
    Q: Write a SQL query that shows individuals' vote and zipcode, ordered from lowest to highest age.
    A: SELECT vote, zipcode, age FROM data ORDER BY age ASC </s>
    Q: Write a SQL query that returns white voters' average age for each state color. 
    A:"""

very_restricted_sql = r"""
    start: WS? "SELECT" WS select_expr WS "FROM" WS from_expr [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
    EOS: "</s>"
    select_expr: STAR | select_list
    bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")"
    bool_expr: var "=" value | var ">" value | var "<" value
    from_expr: "data"
    orderby_expr: var_list WS "ASC" | var_list WS "DESC"
    select_list: select_var ("," WS select_var)*
    var_list: var ("," WS var)*
    select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
    var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
    value: NUMBER | "'red'" | "'blue'" | "'white'" | "'black'" | "'latino'" | "'republican'" | "'democrat'" | "'male'" | "'female'"
    STAR: "*"
    NUMBER: /\d+/
    WS: /[ ]/
"""

In [4]:
from genparse.lm import AsyncGreedilyTokenizedLLM
genparse_llm = AsyncGreedilyTokenizedLLM.from_name("codellama/CodeLlama-7b-Instruct-hf", batch_size=40)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from genparse.cfglm import EarleyBoolMaskCFGLM
from genparse.util import LarkStuff
guide = EarleyBoolMaskCFGLM(LarkStuff(very_restricted_sql).char_cfg(.99, ignore='[ ]?'))

from genparse.steer import HFPPLSampler
sampler = HFPPLSampler(llm=genparse_llm, guide=guide)

from genparse.proposal import TokenProposal, CharacterProposal
proposal = CharacterProposal(llm=genparse_llm, guide=guide)

testing a modified version

In [10]:
particle_approx, record = sampler.run_inference(
    prompt = prompt,
    proposal = proposal,
    method = "smc-standard-record", # use version of smc that keeps a record
    n_particles = 12,
    max_tokens = 60,
    verbosity = 1
)


├ Particle 0 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 1 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 2 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 3 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 4 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 5 (weight -0.0642). ` ` :  
├ Particle 6 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 7 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 8 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 9 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 10 (weight -0.0642). ` SELECT` :  SELECT
├ Particle 11 (weight -0.0642). ` SELECT` :  SELECT
│ Step 1 average weight: -0.0642
└╼
├ Particle 0 (weight -0.1388). ` state` :  SELECT state
├ Particle 1 (weight -0.1388). ` state` :  SELECT state
├ Particle 2 (weight -0.1388). ` state` :  SELECT state
├ Particle 3 (weight -0.1388). ` state` :  SELECT state
├ Particle 4 (weight -0.1388). ` state` :  SELECT state
├ Particle 5 (weight -2.4643). ` SELECT` :   SELECT
├ Particle 6 (weight -0.1388).

In [11]:
import pandas as pd
rec = pd.DataFrame(record)
rec['token'] = rec['context'].apply(lambda r: [ri[-1] if len(ri)>0 else [] for ri in r ])
# rec

In [12]:
import numpy as np
recs = rec.explode(['context','weight','parent','token']).reset_index(drop=True)
N_PARTS = len(rec['context'][0])
recs['particle'] = recs.groupby(recs.index // N_PARTS).cumcount()
recs['context_string'] = recs['context'].apply(lambda r: ''.join(r[:-1]))
recs['float parent'] = recs['parent'].apply(lambda r: float(r))
recs['exp_average weight'] = recs['average weight'].apply(lambda r: np.exp(r))
recs['exp_weight'] = recs['weight'].apply(lambda r: np.exp(r))
recs['prop_exp_weight'] = recs['exp_weight'] / recs['exp_average weight']

In [13]:
particle_approx.posterior


key,value
"SELECT state_color, AVG(age) FROM data WHERE race_ethnicity='white' GROUP BY state_color ORDER BY state_color, age ASC </s>▪",1.0


-----------

plotting quick start


In [14]:
import plotly.express as px
import plotly.graph_objects as go

In [30]:
fig = px.scatter(
    recs, x="step", y="particle", size='prop_exp_weight', color="float parent", hover_name="token", 
    hover_data=["context_string","weight", "resample?"], 
    # animation_frame="step", animation_group="parent", 
    range_x=[20,41]
)
resample_steps = recs[recs['resample?'] == True]['step'].unique()
for step in resample_steps:
    fig.add_vline(x=step, line_width=4, opacity=0.15, line_color="gray")


# Add lines connecting dots with the same parent in consecutive steps
parents = recs['parent'].unique()
steps = sorted(recs['step'].unique())

# Ensure color consistency by creating a color map
available_colors = px.colors.qualitative.Plotly
parents = recs['parent'].unique()
color_map = {parent: available_colors[i % len(available_colors)] for i, parent in enumerate(parents)}

# Filter data for steps where resample? is True
resample_data = recs[recs['resample?'] == True]

# Add lines connecting dots with the same parent at resample steps
for parent in parents:
    parent_data = recs[recs['parent'] == parent]
    
    for step in resample_steps:
        data_step_n = parent_data[parent_data['step'] == step]
        data_step_n1 = parent_data[parent_data['step'] == step - 1]
        
        for _, row_n in data_step_n.iterrows():
            for _, row_n1 in data_step_n1.iterrows():
                fig.add_trace(go.Scatter(
                    x=[row_n['step'], row_n1['step']],
                    y=[row_n['particle'], row_n1['particle']],
                    mode='lines',
                    line=dict(color=color_map[parent]),
                    name=f'parent_{parent}',
                    showlegend=False
                ))

# Show the plot
fig.show()